# Implementation of chatbot using NLP

In [1]:
### Cell 1: Install Dependencies & Setup (Corrected)
!pip install nltk scikit-learn --quiet

import nltk
import random
import ssl  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# SSL Certificate Fix
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Download NLTK Resources
nltk.download(['punkt', 'punkt_tab'], quiet=True)

True

In [2]:
### Cell 2: Intent Definitions
intents = [
    {
        'tag': 'greeting',
        'patterns': ['Hello', 'Hi', 'Hey', 'Greetings', 'Good morning', 'Good afternoon', 'What\'s up'],
        'responses': ['Hello!', 'Hi there!', 'Hey!', 'Good to see you!', 'What\'s up!']
    },
    {
        'tag': 'farewell',
        'patterns': ['Goodbye', 'Bye', 'See you later', 'Farewell', 'Take care', 'Catch you later'],
        'responses': ['Goodbye!', 'See you later!', 'Take care!', 'Farewell!', 'Catch you later!']
    },
    {
        'tag': 'gratitude',
        'patterns': ['Thank you', 'Thanks', 'Appreciate it', 'Thank you very much', 'Much obliged'],
        'responses': ['You’re welcome!', 'No problem!', 'My pleasure!', 'Anytime!', 'You are very welcome!']
    },
    {
        'tag': 'identity',
        'patterns': ['What is your name?', 'Who are you?', 'Tell me your name', 'Your identity'],
        'responses': ['I am a helpful assistant.', 'You can call me AI Assistant.', 'I am here to assist you.', 'I am your digital companion.']
    },
    {
        'tag': 'assistance',
        'patterns': ['Help', 'I need help', 'Can you help me?', 'What can you do?', 'Assist me'],
        'responses': ['I can answer questions and provide information.', 'How can I assist you?', 'I am here to help.', 'I can assist you with many tasks.']
    },
    {
        'tag': 'order_status',
        'patterns': ['What is my order status?', 'Check my order', 'Where is my order?', 'Track order'],
        'responses': ['Please provide your order number.', 'I need your order number to check the status.', 'You can check your order status on our website.', 'To track your order, please provide the tracking number.']
    },
    {
        'tag': 'product_info',
        'patterns': ['Tell me about your products', 'What products do you have?', 'Product information', 'Product details'],
        'responses': ['We offer a variety of products. Please specify which product you are interested in.', 'You can find details on our website.', 'We have a wide range of products available.']
    },
    {
        'tag': 'contact',
        'patterns': ['How can I contact you?', 'Contact information', 'Phone number', 'Email', 'Reach you'],
        'responses': ['You can email us at support@example.com or call us at 555-123-4567.', 'Our contact details are on our website.', 'You can reach us by phone or email.']
    },
    {
        'tag': 'weather',
        'patterns': ['What is the weather like?', 'Weather forecast', 'Is it raining today?', 'Weather report'],
        'responses': ['I cannot provide real-time weather. Please check a weather app.', 'For weather, please check a local weather service.', 'Weather information is available on many online services.']
    },
    {
        'tag': 'recommendation',
        'patterns': ['Recommend a book', 'Suggest a movie', 'What should I watch', 'Suggest a song'],
        'responses': ['For a good read, try "The Hitchhiker\'s Guide to the Galaxy".', 'I recommend "Inception" for a great movie.', 'You might enjoy the series "The Queen\'s Gambit".', 'I suggest listening to "Bohemian Rhapsody" by Queen.']
    },
    {
        'tag': 'definition',
        'patterns': ['What is [word]?', 'Define [word]', 'Meaning of [word]', 'Explain [word]'],
        'responses': ['Please specify the word you want defined.', 'I need a word to provide a definition.', 'Please provide the word to be explained.']
    },
    {
        'tag': 'travel_advice',
        'patterns': ['Travel tips', 'Best places to visit', 'How to plan a trip', 'Travel guide'],
        'responses': ['Plan your itinerary in advance and book accommodations early.', 'Consider visiting cultural landmarks and trying local cuisine.', 'Research local customs to ensure a smooth trip.', 'Use online travel guides for more information.']
    },
    {
        'tag': 'coding_help',
        'patterns': ['How to fix [code error]', 'Coding advice', 'What is [programming concept]', 'Coding tips'],
        'responses': ['Please provide more details about the error.', 'For coding advice, specify the programming language.', 'To explain a programming concept, please be more specific.', 'Coding tips are available online.']
    },
    {
        'tag': 'recipe',
        'patterns': ['Recipe for [food]', 'How to cook [food]', 'Cooking instructions for [food]', 'Make [food]'],
        'responses': ['Please specify the food you want a recipe for.', 'I can find recipes online.', 'Provide the food you would like to make.']
    },
    {
        'tag': 'time',
        'patterns': ['What time is it?', 'Tell me the time', 'Current time', 'Time now'],
        'responses': ['I cannot provide real-time information. Please check your device.', 'Please check your device for the current time.']
    },
    {
        'tag': 'fact',
        'patterns': ['Tell me a fact', 'Give me a random fact', 'Interesting fact', 'Fun fact'],
        'responses': ['Did you know that the Eiffel Tower can be 15 cm taller during the summer?', 'Honey never spoils. Archaeologists have found edible honey in ancient Egyptian tombs.', 'The human brain generates more electrical impulses in a single day than all the telephones in the world combined.']
    },
    {
        'tag': 'health_advice',
        'patterns': ['Health tips', 'How to stay healthy', 'Fitness advice', 'Diet tips'],
        'responses': ['Maintain a balanced diet and exercise regularly.', 'Get enough sleep and manage stress.', 'Stay hydrated and eat nutrient-rich foods.', 'Consult a healthcare professional for personalized advice.']
    },
    {
        'tag': 'news',
        'patterns': ['Latest news', 'What\'s happening', 'News updates', 'Current events'],
        'responses': ['I cannot provide real-time news updates. Please check a news website.', 'For the latest news, please refer to a news source.', 'News updates are available from various online resources.']
    },
    {
        'tag': 'translation',
        'patterns': ['Translate [text] to [language]', 'How to say [text] in [language]', 'Language translation'],
        'responses': ['I cannot provide real-time translation. Please use an online translation tool.', 'For accurate translations, please use a dedicated translation service.']
    },
    {
        'tag': 'learning_resources',
        'patterns': ['Learn about [topic]', 'Educational resources', 'How to study [topic]', 'Online courses'],
        'responses': ['There are many online resources available for learning. Consider platforms like Coursera or Khan Academy.', 'For educational resources, search for reputable online learning platforms.', 'Learning about a specific topic requires focused study and practice.']
    },
    {
        'tag': 'confusion',
        'patterns': [''],
        'responses': ['I’m not sure I understand. Could you please rephrase?', 'Sorry, I don’t understand. Please try again.']
    }
]

In [3]:
def preprocess_data(intents):
    all_patterns = []
    all_tags = []
    for intent in intents:
        for pattern in intent['patterns']:
            # Convert to lowercase and tokenize
            tokens = nltk.word_tokenize(pattern.lower())
            all_patterns.append(' '.join(tokens))
            all_tags.append(intent['tag'])
    return all_patterns, all_tags

patterns, tags = preprocess_data(intents)

In [4]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(patterns)
y = tags

In [5]:
model = LogisticRegression(max_iter=1000)
model.fit(X, y)

LogisticRegression(max_iter=1000)

In [6]:
def get_chatbot_response(user_input):
    # Preprocess input
    processed_input = ' '.join(nltk.word_tokenize(user_input.lower()))
    
    # Vectorize and predict
    vectorized_input = vectorizer.transform([processed_input])
    predicted_tag = model.predict(vectorized_input)[0]
    
    # Find appropriate response
    for intent in intents:
        if intent['tag'] == predicted_tag:
            return random.choice(intent['responses'])
    
    # Fallback response
    return "I'm still learning. Could you rephrase that?"

In [7]:
# Test sample inputs
test_phrases = [
    "Hello!",
    "What's the weather like?",
    "Tell me a fun fact",
    "How do I fix a Python error?"
]

for phrase in test_phrases:
    print(f"User: {phrase}")
    print(f"Bot: {get_chatbot_response(phrase)}\n")

User: Hello!
Bot: Good to see you!

User: What's the weather like?
Bot: For weather, please check a local weather service.

User: Tell me a fun fact
Bot: Honey never spoils. Archaeologists have found edible honey in ancient Egyptian tombs.

User: How do I fix a Python error?
Bot: Coding tips are available online.



In [ ]:
print("Chatbot: Hi! I'm your assistant. Type 'exit' to end the chat.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Chatbot: Goodbye! Have a great day!")
        break
    print("Chatbot:", get_chatbot_response(user_input))

Chatbot: Hi! I'm your assistant. Type 'exit' to end the chat.


You:  good morning


Chatbot: Hey!


You:  give me health advice


Chatbot: Maintain a balanced diet and exercise regularly.


...

Chatbot: There are many online resources available for learning. Consider platforms like Coursera or Khan Academy.


You:  ok thanks


Chatbot: My pleasure!


You:  bye bye


Chatbot: Catch you later!
